In [2]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md5 = raw_input("Do you wanna run md5 if missing? (md5/none)")


ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
raw_bucket = tibanna.s3.raw_file_bucket
out_bucket = tibanna.s3.outfile_bucket

file_url = '/search/?status=uploading&status=upload%20failed&type=FileFastq'
# file_url = '/search/?type=FileReference'
# file_url = '/search/?status=uploading&status=upload%20failed&type=FileProcessed'
file_url = '/search/?lab.display_title=Mitchell%20Guttman%2C%20CALTECH&type=FileProcessed'
all_files = ff_utils.get_metadata(file_url, connection=ff)['@graph']

print len(all_files)


printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            #print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter


    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    
    
    if 'FileProcessed' in file_resp['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket
            
            
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], my_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue
    md5 = False
    
    file_id = file_resp['accession']
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    
    if file_resp.get('md5sum'):
        md5 = True
        
    # is there md5sum for gzip
    if not file_resp.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
        
    # Check workflows f
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status') 

#     print file_resp['accession']    
#     print file_resp['lab']['@id']
#     print "status", file_resp['status']
#     print "md5_status", md5_status
#     print "md5 okay?", md5
#     print file_resp['aliases']

    if status in ["uploading", "upload failed"] and md5_status != 'complete':
        
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            code_md5= "invoke run_md5 " + env + " " + file_resp['display_title'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(30)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id
            
        print 'md5 run missing for', file_id
    
    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print "ERROR:",file_id ,"There is a successful md5 run but status not switched"
        
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            code_md5= "invoke run_md5 " + env + " " + file_resp['display_title'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(30)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id
        
    elif md5_status != 'complete':
        print status, 'file does not have md5 run'
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            code_md5= "invoke run_md5 " + env + " " + file_resp['display_title'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(30)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id
            
    print



Do you wanna run md5 if missing? (md5/none)md5
33
md5 running for 4DNFIGF6TA9O
SUBMIT4DN fdnDCIC TOOLS ARE DEPRECIATED.
      PLEASE SWITCH TO USING DCICUTILS INSTEAD.
      fdnDCIC WILL BE DELETED JUNE 2018.
about to start run md5_4DNFIGF6TA9O.csv8221e362-964f-4378-8b62-9bb1fc1ac6f5
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 23, 13, 5, 43, 31000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '86e14a40-5eab-11e8-abd7-9b5b4c930630', 'HTTPHeaders': {'x-amzn-requestid': '86e14a40-5eab-11e8-abd7-9b5b4c930630', 'content-length': '165', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:md5_4DNFIGF6TA9O.csv8221e362-964f-4378-8b62-9bb1fc1ac6f5'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:md5_4DNFIGF6TA9O.csv8221e362-964f-4378-8b

KeyboardInterrupt: 